In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [8]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets.heloc import HelocDataset
from counterfactuals.datasets.moons import MoonsDataset
from counterfactuals.datasets.law import LawDataset

from counterfactuals.optimizers.approach_gen_disc import ApproachGenDisc

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [4]:
dataset = MoonsDataset(file_path="../data/origin/moons.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

In [6]:
# clf = LogisticRegression()
clf = MLPClassifier((128, 64), max_iter=100)
clf.fit(dataset.X_train, dataset.y_train)
y_pred_train = clf.predict(dataset.X_train)
y_pred_test = clf.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_test, output_dict=False))

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       797
         1.0       1.00      1.00      1.00       842

    accuracy                           1.00      1639
   macro avg       1.00      1.00      1.00      1639
weighted avg       1.00      1.00      1.00      1639



In [9]:
flow = MaskedAutoregressiveFlow(features=2, hidden_features=4, context_features=None)

cf = ApproachGenDisc(gen_model=flow, disc_model=clf)

In [10]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=100,
    verbose=True
)

Epochs:   1%|          | 1/100 [00:00<00:54,  1.82it/s]

Epoch 0, Train: 3.059175865990775, test: 1.8761619329452515


Epochs:  11%|█         | 11/100 [00:05<00:45,  1.96it/s]

Epoch 10, Train: 1.5472110133150438, test: 1.5468325477380018


Epochs:  21%|██        | 21/100 [00:10<00:40,  1.96it/s]

...

Epochs:  81%|████████  | 81/100 [00:41<00:09,  1.96it/s]

Epoch 80, Train: 1.3099558632095138, test: 1.334484957731687


Epochs:  91%|█████████ | 91/100 [00:46<00:04,  1.97it/s]

Epoch 90, Train: 1.2869224378040858, test: 1.297770124215346


Epochs: 100%|██████████| 100/100 [00:51<00:00,  1.96it/s]


In [11]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       0.49      1.00      0.65       797
         1.0       0.00      0.00      0.00       842

    accuracy                           0.49      1639
   macro avg       0.24      0.50      0.33      1639
weighted avg       0.24      0.49      0.32      1639



/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [18]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=100, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

100it [00:26,  3.75it/s]


In [19]:
y_cfs_pred_mlpc = clf.predict(Xs_cfs)
y_orig_pred_mlpc = clf.predict(dataset.X_test[:100])

np.sum(y_cfs_pred_mlpc != y_orig_pred_mlpc) / y_orig_pred_mlpc.shape[0]

0.11